In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import random
from tqdm import tqdm
import esm

In [2]:
seed_value = 42
random.seed(seed_value)                         # 设置 random 模块的随机种子
np.random.seed(seed_value)                      # 设置 numpy 模块的随机种子
torch.manual_seed(seed_value)                   # 设置 PyTorch 中 CPU 的随机种子
#tf.random.set_seed(seed_value)                 # 设置 Tensorflow 中随机种子
if torch.cuda.is_available():                   # 如果可以使用 CUDA，设置随机种子
    torch.cuda.manual_seed(seed_value)          # 设置 PyTorch 中 GPU 的随机种子
# 检测GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 加载ESM模型

In [3]:
# Load ESM-2 model
esm_model, alphabet = esm.pretrained.esm2_t6_8M_UR50D() #320d
batch_converter = alphabet.get_batch_converter()
esm_model.eval()  # disables dropout for deterministic results

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=120, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((320,), eps=1e-05, elementwis

In [4]:
def ESM_feature(sequence):
    data = []
    for i in tqdm(sequence):
        row = (i,i)
        data.append(row)

    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]
# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
    return sequence_representations

In [43]:
#df=pd.read_csv("test192.txt", na_filter=False)
df=pd.read_excel("D:\Desk\BioPeptide_model\input_data\Peony.xlsx", na_filter=False)

In [44]:
df.head(5)

,Peptide
0,YTPDYKTK
1,VVAPPERKY
2,FAGDDAPR
3,GFAGDDAPR
4,RVTPQPGVPPEE


In [9]:
df.shape

(31, 1)

In [11]:
len_num = [len(num) for num in df["Peptide"]]
df["len_num"] = len_num

In [12]:
RF_sequence = df[df["len_num"]>10]["Peptide"]
transformer_sequence = df[df["len_num"]<11]["Peptide"]

In [13]:
RF_sequence_representations = ESM_feature(RF_sequence)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [14]:
transformer_sequence_representations = ESM_feature(transformer_sequence)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<?, ?it/s]


# 分别产生特征和模型

In [47]:
import joblib
scaler_filename =r"model\Standard_scaler.save"
scaler = joblib.load(scaler_filename)

In [16]:
RF_pred = torch.stack(RF_sequence_representations)

In [17]:
transformer_X_pred = torch.stack(transformer_sequence_representations)

In [18]:
RF_X_pred = scaler.transform(RF_pred)

In [19]:
transformer_X_pred = scaler.transform(transformer_X_pred)

## 加载预测模型

In [20]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(TransformerModel, self).__init__()
        # 构建Transformer编码层，参数包括输入维度、注意力头数
        # 其中d_model要和模型输入维度相同
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size,  # 输入维度
                                                        batch_first=True,
                                                        nhead=8)             # 注意力头数
        # 构建Transformer编码器，参数包括编码层和层数
        self.encoder = nn.TransformerEncoder(self.encoder_layer,             # 编码层
                                             num_layers=10)                   # 层数
        # 构建线性层，参数包括输入维度和输出维度（num_classes）
        self.fc = nn.Linear(input_size,                                      # 输入维度
                            num_classes)                                     # 输出维度


    def forward(self, x):
        #print("A:", x.shape)  # torch.Size([142, 13])
        x = x.unsqueeze(1)    # 增加一个维度，变成(batch_size, 1, input_size)的形状
        #print("B:", x.shape)  # torch.Size([142, 1, 13])
        x = self.encoder(x)   # 输入Transformer编码器进行编码
        #print("C:", x.shape)  # torch.Size([142, 1, 13])
        x = x.squeeze(1)      # 压缩第1维，变成(batch_size, input_size)的形状
        #print("D:", x.shape)  # torch.Size([142, 13])
        x = self.fc(x)        # 输入线性层进行分类预测
        #print("E:", x.shape)  # torch.Size([142, 3])
        return x

In [21]:
RF_model = joblib.load(r"model\RandomForest_classf.pkl")

D:\BaiduNetdiskDownload\Anaconda\envs\AHTPeptideFusion\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\BaiduNetdiskDownload\Anaconda\envs\AHTPeptideFusion\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
model = torch.load(r'model\transformer_class.pt',map_location=torch.device('cuda:0'))#.to(device)

# 模型预测

In [23]:
RF_y = RF_model.predict(RF_pred)
RF_y_pro = RF_model.predict_proba(RF_pred)

In [24]:
RF_df = pd.DataFrame(np.array(RF_sequence),columns=["Peptide"])
RF_df["Active_pro"] = RF_y_pro[:,0]
RF_df["Inactive_pro"] = RF_y_pro[:,1]

In [25]:
#extraT_df.to_excel("RF_ACE_true_prediction_result.xlsx",index=None)

In [26]:
#svc_df.to_excel("4_ACE_true_prediction_result.xlsx",index=None)

In [27]:
with torch.no_grad():
    model.eval()
    y_hat = model(torch.tensor(transformer_X_pred).float().to(device))   # 使用训练好的模型对测试集进行预测
    y_score = torch.softmax(y_hat, dim=1).data.cpu().numpy()
    prediction = torch.max(F.softmax(y_hat,dim=1), 1)[1]
    #pred_y = prediction.data.cpu().numpy().squeeze()

In [28]:
transformer_df = pd.DataFrame(np.array(transformer_sequence),columns=["Peptide"])
transformer_df["Active_pro"] = y_score[:,0]
transformer_df["Inactive_pro"] = y_score[:,1]

In [29]:
#transformer_df.to_excel("prediction_result.xlsx",index=None)

In [30]:
df_all = pd.concat([RF_df,transformer_df], axis=0)

In [31]:
#df_all

,Peptide,Active_pro,Inactive_pro
0,RVTPQPGVPPEE,0.920000,0.080000
1,SSDTIDNVKAK,0.880000,0.120000
2,VAPEEHPVLLTEAPLNPK,0.910000,0.090000
3,VVEEPVSSPEPK,0.900000,0.100000
4,AGFAGDDAPRA,0.890000,0.110000
0,YTPDYKTK,0.921637,0.078363
1,VVAPPERKY,0.999989,0.000011
2,FAGDDAPR,0.646214,0.353786
3,GFAGDDAPR,0.571227,0.428772
4,SGPSIVH,0.130866,0.869134


## 保存结果

In [36]:
df_all['ACE_Label'] = df_all['Active_pro'].apply(lambda x: 'Active' if x > 0.5 else 'Non-Active')

In [37]:
df_all.head(5)

,Peptide,Active_pro,Inactive_pro,Label,ACE_Label
0,RVTPQPGVPPEE,0.92,0.08,Active,Active
1,SSDTIDNVKAK,0.88,0.12,Active,Active
2,VAPEEHPVLLTEAPLNPK,0.91,0.09,Active,Active
3,VVEEPVSSPEPK,0.90,0.10,Active,Active
4,AGFAGDDAPRA,0.89,0.11,Active,Active


In [87]:
#df_all.to_excel("ACE_peptide_result.xlsx",index=None)

In [40]:
df_all_ok = df_all[["Peptide","ACE_Label"]]

In [42]:
df_all_ok.to_excel("D:\Desk\BioPeptide_model\output_result\ACE_peptide_result.xlsx",index=None)

In [32]:
#sum([1 for x in df_all["Active_pro"] if x > 0.5])

6